# Assignment 5

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 72

import pandas as pd
import numpy as np
import seaborn as sns


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv("./heart_dataset.csv")
df.head()

Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1              N      1.0     Flat             1  
2              N      0.0       Up             0  
3              Y      1.5     Flat             1  
4              N      0.0       Up             0

In [3]:
df['ChestPainType'].unique()

array(['ATA', 'NAP', 'ASY', 'TA'], dtype=object)

In [4]:
df['RestingECG'].unique()

array(['Normal', 'ST', 'LVH'], dtype=object)

In [5]:
df['ST_Slope'].unique()

array(['Up', 'Flat', 'Down'], dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [7]:
# One-hot encoding the 3 columns logged above

columns_to_encode = ['RestingECG', 'ChestPainType', "ExerciseAngina", "Sex"]
df = pd.get_dummies(df, columns=columns_to_encode)

df.head()

# df = df.drop(['ST_Slope', 'RestingECG', 'ChestPainType'], axis = 1)



Age Sex  RestingBP  Cholesterol  FastingBS  MaxHR  Oldpeak ST_Slope  \
0   40   M        140          289          0    172      0.0       Up   
1   49   F        160          180          0    156      1.0     Flat   
2   37   M        130          283          0     98      0.0       Up   
3   48   F        138          214          0    108      1.5     Flat   
4   54   M        150          195          0    122      0.0       Up   

   HeartDisease  RestingECG_LVH  RestingECG_Normal  RestingECG_ST  \
0             0               0                  1              0   
1             1               0                  1              0   
2             0               0                  0              1   
3             1               0                  1              0   
4             0               0                  1              0   

   ChestPainType_ASY  ChestPainType_ATA  ChestPainType_NAP  ChestPainType_TA  \
0                  0                  1                  0                 0   
1                  0                  0                  1                 0   
2                  0                  1                  0                 0   
3                  1                  0                  0                 0   
4                  0                  0                  1                 0   

   ExerciseAngina_N  ExerciseAngina_Y  
0                 1                 0  
1                 1                 0  
2                 1                 0  
3                 0                 1  
4                 1                 0

In [8]:
st_order = ['Down', 'Flat', 'Up']

ordinal_encoder = OrdinalEncoder(categories=[st_order])

df['ST_Slope'] = ordinal_encoder.fit_transform(df[['ST_Slope']])
    
df.head()

Age Sex  RestingBP  Cholesterol  FastingBS  MaxHR  Oldpeak  ST_Slope  \
0   40   M        140          289          0    172      0.0       2.0   
1   49   F        160          180          0    156      1.0       1.0   
2   37   M        130          283          0     98      0.0       2.0   
3   48   F        138          214          0    108      1.5       1.0   
4   54   M        150          195          0    122      0.0       2.0   

   HeartDisease  RestingECG_LVH  RestingECG_Normal  RestingECG_ST  \
0             0               0                  1              0   
1             1               0                  1              0   
2             0               0                  0              1   
3             1               0                  1              0   
4             0               0                  1              0   

   ChestPainType_ASY  ChestPainType_ATA  ChestPainType_NAP  ChestPainType_TA  \
0                  0                  1                  0                 0   
1                  0                  0                  1                 0   
2                  0                  1                  0                 0   
3                  1                  0                  0                 0   
4                  0                  0                  1                 0   

   ExerciseAngina_N  ExerciseAngina_Y  
0                 1                 0  
1                 1                 0  
2                 1                 0  
3                 0                 1  
4                 1                 0